Simple Logistic Regression has high accuracy score because of imbalanced data.

In [0]:
lgModel=LogisticRegression()

In [0]:
lgModel.fit(train_dta,Y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
pred_y=lgModel.predict(test_dta)

In [0]:
lgModel.score(test_dta,Y_test)

0.963525682668399

In [0]:
print(confusion_matrix(Y_test,pred_y))

[[172051      0]
 [  6513      0]]


### **f1 score is 0 because the model is heavily biased and not classifying class 1. Model is predicting only class 0 as constant value.**

In [0]:
f1_score(Y_test,pred_y)

0.0

## **LogistcRegression with SMOTE to deal with imbalanced data. There is a nominal increase in f1 score.**

In [0]:
os = SMOTE(random_state=0)
X_train, X_test, y_train, y_test = train_test_split(df_final, target, test_size=0.3, random_state=0)
columns = X_train.columns

In [0]:
os_data_X,os_data_y=os.fit_sample(X_train, y_train)
os_data_X = pd.DataFrame(data=os_data_X,columns=columns )
os_data_y= pd.DataFrame(data=os_data_y,columns=['y'])
# we can Check the numbers of our data
print("length of oversampled data is ",len(os_data_X))
print("Number of no subscription in oversampled data",len(os_data_y[os_data_y['y']==0]))
print("Number of subscription",len(os_data_y[os_data_y['y']==1]))
print("Proportion of no subscription data in oversampled data is ",len(os_data_y[os_data_y['y']==0])/len(os_data_X))
print("Proportion of subscription data in oversampled data is ",len(os_data_y[os_data_y['y']==1])/len(os_data_X))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


length of oversampled data is  802866
Number of no subscription in oversampled data 401433
Number of subscription 401433
Proportion of no subscription data in oversampled data is  0.5
Proportion of subscription data in oversampled data is  0.5


In [0]:
lgModel=LogisticRegression()
lgModel.fit(train_dta,os_data_y)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
lgModel.score(test_dta,y_test)

0.49736789050424496

In [0]:
print(confusion_matrix(y_test,pred_y))

[[85050 87035]
 [ 2717  3762]]


In [0]:
print(classification_report(y_test,pred_y))

              precision    recall  f1-score   support

           0       0.97      0.49      0.65    172085
           1       0.04      0.58      0.08      6479

    accuracy                           0.50    178564
   macro avg       0.51      0.54      0.37    178564
weighted avg       0.94      0.50      0.63    178564



## **LinearSVC Model did fairly well with modified class weights**

In [0]:
model = LinearSVC(dual=True,max_iter=2000,class_weight={0:1,1:10})

In [0]:
model.fit(train_dta,Y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=1.0, class_weight={0: 1, 1: 10}, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=2000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [0]:
linearsvc_y_pred = model.predict(test_dta)
print(accuracy_score(Y_test,linearsvc_y_pred))

0.9595047153961604


In [0]:
print(confusion_matrix(Y_test,linearsvc_y_pred))

[[171219    841]
 [  6390    114]]


## **XGBoostClassifier**

In [0]:
X_train,X_test,Y_train,Y_test=train_test_split(df_final,target,test_size=0.3,random_state=2)

model = XGBClassifier(scale_pos_weight=10)

skfold=StratifiedKFold(n_splits=10,random_state=2)


roc_auc=cross_val_score(model,X_train,Y_train,scoring='roc_auc',cv=skfold) # after setting scale_pos_weight to 27

print(roc_auc)

[0.63280109 0.64246721 0.64928406 0.65120351 0.6331232  0.64161981
 0.63834732 0.63426045 0.63518893 0.62573243]


In [0]:
model.fit(X_train.values,Y_train.values)

pred_y=model.predict(X_test.values)

In [0]:
#np.unique(pred_y)

print('accuracy score ',accuracy_score(Y_test,pred_y))

print(confusion_matrix(Y_test,pred_y))

print(f1_score(Y_test,pred_y))

accuracy score  0.9524596223202886
[[169786   2274]
 [  6215    289]]
0.06374765633616411


In [0]:
y_probs=model.predict_proba(X_test.values)

In [0]:
y_probs_1=y_probs[:,1]
yprobs1=y_probs_1 > 0.5
yprobs1 = yprobs1 * 1
np.unique(yprobs1,return_counts=True)

(array([0, 1]), array([176001,   2563], dtype=int64))

In [0]:
lr_precision, lr_recall, _=precision_recall_curve(Y_test,y_probs_1)

In [0]:
auc(lr_recall, lr_precision)

0.06332816088157696

# **If not missing positive a priority then XGBoost is the by far the best classifier.**

**LogisticRegressin would do good if not missing **